# Estimativa de sazonalidade

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# Testando com microdados_anac

In [2]:
# Ler a base da anac
bd_anac = pd.read_csv('data/csv/Dados_Estatisticos.csv', sep = ';', skiprows = 1)

In [3]:
# Correção para AEROPORTO_DE_ORIGEM_NOME = JACAREACANGA e AEROPORTO_DE_ORIGEM_SIGLA = SBSI
bd_anac.loc[(bd_anac['AEROPORTO_DE_ORIGEM_NOME'] == 'JACAREACANGA') & 
            (bd_anac['AEROPORTO_DE_ORIGEM_SIGLA'] == 'SBSI'), 'AEROPORTO_DE_ORIGEM_SIGLA'] = 'SWSI'

# Correção para AEROPORTO_DE_DESTINO_NOME = JACAREACANGA e AEROPORTO_DE_DESTINO_SIGLA = SBSI
bd_anac.loc[(bd_anac['AEROPORTO_DE_DESTINO_NOME'] == 'JACAREACANGA') & 
            (bd_anac['AEROPORTO_DE_DESTINO_SIGLA'] == 'SBSI'), 'AEROPORTO_DE_DESTINO_SIGLA'] = 'SWSI'

# Correção para AEROPORTO_DE_ORIGEM_NOME = SINOP e AEROPORTO_DE_ORIGEM_SIGLA = SWSI
bd_anac.loc[(bd_anac['AEROPORTO_DE_ORIGEM_NOME'] == 'SINOP') & 
            (bd_anac['AEROPORTO_DE_ORIGEM_SIGLA'] == 'SWSI'), 'AEROPORTO_DE_ORIGEM_SIGLA'] = 'SBSI'

# Correção para AEROPORTO_DE_DESTINO_NOME = SINOP e AEROPORTO_DE_DESTINO_SIGLA = SWSI
bd_anac.loc[(bd_anac['AEROPORTO_DE_DESTINO_NOME'] == 'SINOP') & 
            (bd_anac['AEROPORTO_DE_DESTINO_SIGLA'] == 'SWSI'), 'AEROPORTO_DE_DESTINO_SIGLA'] = 'SBSI'

# Correção para AEROPORTO_DE_ORIGEM_SIGLA = SSZW
bd_anac.loc[bd_anac['AEROPORTO_DE_ORIGEM_SIGLA'] == 'SSZW', 'AEROPORTO_DE_ORIGEM_SIGLA'] = 'SBPG'

# Correção para AEROPORTO_DE_DESTINO_SIGLA = SSZW
bd_anac.loc[bd_anac['AEROPORTO_DE_DESTINO_SIGLA'] == 'SSZW', 'AEROPORTO_DE_DESTINO_SIGLA'] = 'SBPG'

# Correção para AEROPORTO_DE_ORIGEM_SIGLA = SWRD
bd_anac.loc[bd_anac['AEROPORTO_DE_ORIGEM_SIGLA'] == 'SWRD', 'AEROPORTO_DE_ORIGEM_SIGLA'] = 'SBRD'

# Correção para AEROPORTO_DE_DESTINO_SIGLA = SWRD
bd_anac.loc[bd_anac['AEROPORTO_DE_DESTINO_SIGLA'] == 'SWRD', 'AEROPORTO_DE_DESTINO_SIGLA'] = 'SBRD'

In [4]:
def adicionar_coluna_id_aeroporto_origem(bd_anac):
    bd_anac['ID_AEROPORTO_ORIGEM'] = bd_anac['AEROPORTO_DE_ORIGEM_SIGLA'] + ' - ' + bd_anac['AEROPORTO_DE_ORIGEM_NOME']
    return bd_anac

def adicionar_coluna_id_aeroporto_destino(bd_anac):
    bd_anac['ID_AEROPORTO_DESTINO'] = bd_anac['AEROPORTO_DE_DESTINO_SIGLA'] + ' - ' + bd_anac['AEROPORTO_DE_DESTINO_NOME']
    return bd_anac

def adicionar_coluna_data(bd_anac):
    # Renomear as colunas temporariamente para "year" e "month"
    bd_anac = bd_anac.rename(columns={'ANO': 'year', 'MES': 'month'})
    
    # Adicionar uma coluna "day" com todos os valores iguais a 1
    bd_anac['day'] = 1
    
    # Criar a coluna "DATA" utilizando as colunas "year", "month", "day"
    bd_anac['DATA'] = pd.to_datetime(bd_anac[['year', 'month', 'day']])
    
    # Restaurar os nomes originais das colunas
    bd_anac = bd_anac.rename(columns={'year': 'ANO', 'month': 'MES'})
    
    # Remover a coluna "day"
    bd_anac = bd_anac.drop(columns='day')
    
    return bd_anac

bd_anac = adicionar_coluna_id_aeroporto_origem(bd_anac)
bd_anac = adicionar_coluna_id_aeroporto_destino(bd_anac)
bd_anac = adicionar_coluna_data(bd_anac)

In [5]:
def filtrar_bd_anac(bd_anac):
    # Aplicar filtros
    bd_anac_filtrada = bd_anac[
        (bd_anac['NATUREZA'] == 'DOMÉSTICA') & 
        (bd_anac['GRUPO_DE_VOO'] == 'REGULAR') &
        (bd_anac['PASSAGEIROS_PAGOS'] > 0)
    ]
    
    # Selecionar colunas desejadas
    colunas_selecionadas = [
        'DATA', 'EMPRESA_SIGLA', 'EMPRESA_NOME', 
        'ID_AEROPORTO_ORIGEM', 'AEROPORTO_DE_ORIGEM_SIGLA', 
        'AEROPORTO_DE_ORIGEM_UF', 'ID_AEROPORTO_DESTINO', 
        'AEROPORTO_DE_DESTINO_SIGLA', 'AEROPORTO_DE_DESTINO_UF', 
        'PASSAGEIROS_PAGOS', 'CARGA_PAGA_KG'
    ]
    
    # Retornar o DataFrame filtrado com as colunas selecionadas
    return bd_anac_filtrada[colunas_selecionadas]

# Exemplo de uso
bd_anac_filtrada = filtrar_bd_anac(bd_anac)

In [6]:
aeroportos_piar = ['SBTE - TERESINA', 'SBBR - BRASÍLIA', 'SBPB - PARNAÍBA', 'SWKQ - SÃO RAIMUNDO NONATO', 'SNBR - BARREIRAS', 'SBFZ - FORTALEZA', 'SBSL - SÃO LUÍS']

In [10]:
def criar_sazonalidade_piar(bd_anac, aeroportos_piar):
    # Filtrar registros onde ID_AEROPORTO_ORIGEM ou ID_AEROPORTO_DESTINO está na lista
    origem = bd_anac[bd_anac['ID_AEROPORTO_ORIGEM'].isin(aeroportos_piar)].copy()
    destino = bd_anac[bd_anac['ID_AEROPORTO_DESTINO'].isin(aeroportos_piar)].copy()

    # Adicionar coluna 'ds_aeroporto' para origem e destino
    origem['ds_aeroporto'] = origem['ID_AEROPORTO_ORIGEM']
    destino['ds_aeroporto'] = destino['ID_AEROPORTO_DESTINO']

    # Selecionar e renomear colunas
    colunas_selecionadas = ['ds_aeroporto', 'DATA', 'PASSAGEIROS_PAGOS']
    origem = origem[colunas_selecionadas]
    destino = destino[colunas_selecionadas]

    # Concatenar origem e destino
    sazonalidade_piar = pd.concat([origem, destino])

    # Renomear colunas
    sazonalidade_piar = sazonalidade_piar.rename(columns={
        'DATA': 'dt_registro',
        'PASSAGEIROS_PAGOS': 'nr_passag_pagos'
    })

    # Agrupar por 'ds_aeroporto' e 'dt_registro', somando 'nr_passag_pagos'
    sazonalidade_piar = sazonalidade_piar.groupby(
        ['ds_aeroporto', 'dt_registro'], as_index=False
    ).agg({'nr_passag_pagos': 'sum'})

    return sazonalidade_piar

# Chamar a função para criar o DataFrame
sazonalidade_piar = criar_sazonalidade_piar(bd_anac, aeroportos_piar)

In [13]:
def decomposicao_multiplicativa(aeroporto, sazonalidade_piar):
    # Filtrar os dados para o aeroporto específico
    df_aeroporto = sazonalidade_piar[sazonalidade_piar['ds_aeroporto'] == aeroporto].copy()
    
    # Garantir que 'dt_registro' esteja no formato datetime
    df_aeroporto['dt_registro'] = pd.to_datetime(df_aeroporto['dt_registro'])
    
    # Agrupar os dados por mês, somando 'nr_passag_pagos'
    df_mensal = df_aeroporto.set_index('dt_registro').resample('M').sum()
    
    # Remover valores nulos e valores zero ou negativos
    df_mensal = df_mensal[df_mensal['nr_passag_pagos'] > 0].dropna()
    
    # Verificar se há dados suficientes
    if len(df_mensal) < 24:
        print(f"Dados insuficientes para o aeroporto {aeroporto}. É necessário pelo menos 24 observações mensais com valores positivos.")
        return None
    
    # Decomposição multiplicativa
    decomposicao_multiplicativa = seasonal_decompose(df_mensal['nr_passag_pagos'], model='multiplicative', period=12)
    
    # Retornar os valores sazonais multiplicativos
    return decomposicao_multiplicativa.seasonal

In [16]:
# Inicializar um DataFrame vazio para armazenar os indicadores sazonais
indicadores_sazonais_piar = pd.DataFrame()

# Analisar cada aeroporto e obter os valores sazonais multiplicativos
for aeroporto in aeroportos_piar:
    print(f"Analisando o aeroporto: {aeroporto}")
    valores_sazonais = decomposicao_multiplicativa(aeroporto, sazonalidade_piar)
    
    if valores_sazonais is not None:
        # Converter os valores sazonais em DataFrame
        df_sazonais = valores_sazonais.reset_index()
        df_sazonais.columns = ['dt_registro', 'valor_sazonal']
        
        # Adicionar o mês como uma nova coluna (dois dígitos)
        df_sazonais['mes'] = df_sazonais['dt_registro'].dt.strftime('%m')
        
        # Adicionar o aeroporto como uma coluna
        df_sazonais['ds_aeroporto'] = aeroporto
        
        # Agrupar por aeroporto e mês, somando os valores sazonais
        df_resumo = df_sazonais.groupby(['ds_aeroporto', 'mes'], as_index=False)['valor_sazonal'].mean()
        
        # Concatenar com o DataFrame principal
        indicadores_sazonais_piar = pd.concat([indicadores_sazonais_piar, df_resumo], ignore_index=True)

Analisando o aeroporto: SBTE - TERESINA
Analisando o aeroporto: SBBR - BRASÍLIA
Analisando o aeroporto: SBPB - PARNAÍBA
Analisando o aeroporto: SWKQ - SÃO RAIMUNDO NONATO
Dados insuficientes para o aeroporto SWKQ - SÃO RAIMUNDO NONATO. É necessário pelo menos 24 observações mensais com valores positivos.
Analisando o aeroporto: SNBR - BARREIRAS
Analisando o aeroporto: SBFZ - FORTALEZA
Analisando o aeroporto: SBSL - SÃO LUÍS


In [17]:
indicadores_sazonais_piar.to_excel('data/excel/xlsx/sazonalidade_piar.xlsx', index=False)